In [2]:
import os

In [3]:
mpararel_folder = "mpararel_clean"

In [4]:
relations = [x.replace(".jsonl", "") for x in os.listdir(os.path.join(
    mpararel_folder, "en/triples"))]
langs = os.listdir(mpararel_folder)


In [5]:
import json

In [6]:
def get_total_phrases(mpararel_language_folder):
    total_phrases = 0
    for relation in relations:
        with open(os.path.join(mpararel_language_folder, "triples", relation + ".jsonl")) as f_triples:
            triples_count = sum([1 for line in f_triples])
        with open(os.path.join(mpararel_language_folder, "patterns", relation + ".jsonl")) as f_patterns:
            valid_patterns = 0
            for line in f_patterns:
                data = json.loads(line)
                if (data["pattern"].count("[X]") == 1 and
                        data["pattern"].count("[Y]") == 1):
                    valid_patterns += 1
            if valid_patterns <= 1:
                #print("Warning: this relation ({}) should be dropped since it contains one or less patterns".format(relation))
                continue
        total_phrases += valid_patterns * triples_count
    return total_phrases

lang_to_phrases_count = {}
for lang in langs:
    print("Reading lang:", lang)
    lang_to_phrases_count[lang] = get_total_phrases(
        os.path.join(mpararel_folder, lang))

Reading lang: ar
Reading lang: tr
Reading lang: lt
Reading lang: hu
Reading lang: ast
Reading lang: sl
Reading lang: hr
Reading lang: min
Reading lang: lmo
Reading lang: en
Reading lang: new
Reading lang: no
Reading lang: sk
Reading lang: ta
Reading lang: ga
Reading lang: gl
Reading lang: nl
Reading lang: sw
Reading lang: kn
Reading lang: ba
Reading lang: ceb
Reading lang: hy
Reading lang: uz
Reading lang: sq
Reading lang: ro
Reading lang: ca
Reading lang: cs
Reading lang: war
Reading lang: pl
Reading lang: ur
Reading lang: it
Reading lang: cv
Reading lang: zh
Reading lang: ht
Reading lang: bar
Reading lang: sr
Reading lang: bpy
Reading lang: ms
Reading lang: ce
Reading lang: be
Reading lang: an
Reading lang: el
Reading lang: kk
Reading lang: fi
Reading lang: et
Reading lang: io
Reading lang: azb
Reading lang: vo
Reading lang: fa
Reading lang: ja
Reading lang: ne
Reading lang: yo
Reading lang: fr
Reading lang: my
Reading lang: vi
Reading lang: eu
Reading lang: es
Reading lang: he
Readi

In [9]:
import pandas as pd

In [11]:
df = pd.DataFrame(lang_to_phrases_count.items(), columns=['lang', 'phrases_count'])

In [21]:
en_total = df[df['lang'] == 'en']['phrases_count'].values[0]
df['phrases_count'] = df['phrases_count']/en_total
df

,lang,phrases_count
0,ar,0.477289
1,tr,0.438437
2,lt,0.266388
3,hu,0.546281
4,ast,0.000000
...,...,...
99,nn,0.000000
100,uk,0.481269
101,ru,0.646260
102,scn,0.000000


In [23]:
len(df)

104

In [26]:
len(df[df['phrases_count'] > 0.2])

48

In [28]:
valid_langs = df[df['phrases_count'] > 0.2]['lang'].values

In [29]:
for lang in valid_langs:
    print(">>>>>>>>>>>>>>>>>>>>>")
    print(lang)
    for relation in relations:
        print("--- {} ---".format(relation))
        with open(os.path.join(mpararel_folder, lang, "patterns", relation + ".jsonl")) as f_patterns:
            for line in f_patterns:
                data = json.loads(line)
                if (data["pattern"].count("[X]") == 1 and
                        data["pattern"].count("[Y]") == 1):
                    print(data["pattern"])
        print()

>>>>>>>>>>>>>>>>>>>>>
ar
--- P30 ---
[X] موجود في [Y] .
[X] يقع في القارة [Y] .
[X] ينتمي إلى قارة [Y] .
[X] هو جزء من قارة [Y].

--- P279 ---
[X] هو فرعية من [Y].
[X]، وهي فئة فرعية من [Y].
[X]، فئة فرعية من [Y].
[X]، نوع من [Y].

--- P39 ---
[X] لديه موقف [Y] .
[X]، الذي لديه موقف [Y].
[X] ، الذي هو موقف [Y] .
[X] هو [Y] .
[X]، الذي يحمل موقف [Y].
[X] يحمل موقف [Y] .

--- P1303 ---
[X] يلعب [Y] .
[Y] لاعب [X]
[X] يلعب [Y] .
[X] هو لاعب [Y].

--- P19 ---
[X] ولد في [Y] .
[X] في الأصل من [Y] .
[X] كان في الأصل من [Y] .
[X] أصلي إلى [Y].
[X] كان مواطني [Y].
[X] نشأت من [Y] .
[X] ينشأ من [Y] .
[X] كان [Y] شخص.
[X] كان شخص [Y] .
[X] هو شخص [Y].
[X] هو شخص [Y] .
[X]، [Y] - شخص.
[X] ، [Y] شخص.

--- P101 ---
[X] يعمل في مجال [Y] .
[X] متخصص في [Y] .
[Y] هو تخصص [X].
خبرات [X] هي [Y].
[Y] هي خبرة [X].
مجال نشاط [X] هو [Y].
مجال عمل [X] هو [Y].
[Y] هو مجال نشاط [X].
[X] مجال العمل هو [Y].
[X] مجال العمل هو [Y].
[X] مجال النشاط هو [Y].
[X] يعمل في مجال [Y] .

--- P1412 ---
[X] تستخدم للتواصل في